## Dataset:
## Date:

### Set-up

In [1]:
### NAME FILES however you want to save them
raw_file = "DATASET_scanpy_raw_filtered_DATE.h5ad"
results_file = "DATASET_scanpy_DATE.h5ad"

### Load packages
import numpy as np
import pandas as pd
import scanpy as sc

### Load data -- either read in anndata, loom file, or from raw 10x
path = './filtered_gene_bc_matrices/hg19/'     #the path to the folder
adata = sc.read(path + 'matrix.mtx').T    #Transpose data
adata.var_names = pd.read_csv(path +'genes.tsv',header=None,sep='\t')[1]
adata.obs_names = pd.read_csv(path + 'barcodes.tsv',header=None)[0]

adata.var_names_make_unique()

### Check settings
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=80)
sc.logging.print_versions()


/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


AttributeError: module 'scanpy' has no attribute 'settings'

In [ ]:
### Check the loaded data (here named 'adata')
adata

### Preprocess

In [ ]:
### Initial filtering genes and cells
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)


In [ ]:
### Find cells with high % mitochondrial content
mito_genes = [name for name in adata.var_names if name.startswith('MT-')]
adata.obs['percent_mito'] = np.sum(adata[:,mito_genes].X,axis=1).A1 / np.sum(adata.X,axis=1).A1

### Find cells with high counts (potential doublets)
adata.obs['n_counts'] = adata.X.sum(axis=1).A1
sc.pl.violin(adata,[’n_genes’,’n_counts’,’percent_mito’],jitter=0.4,multi_panel=True)


In [ ]:
### Other visualization options:
sc.pl.scatter(adata,x=’n_counts’,y=‘percent_mito’)
sc.pl.scatter(adata,x=’n_counts’,’y=’n_genes’)

import matplotlib.pyplot as plt
import seaborn as sns

sns.distplot(adata.obs[‘percent_mito’],bins=30)
plt.show()
sns.distplot(adata.obs[’n_counts’],bins=30)
plt.show()


In [ ]:
### Remove these cells -- change based on above
adata = adata[adata.obs['n_counts']<7000,:]
adata = adata[adata.obs['percent_mito']<0.15,:]


In [ ]:
### Save raw data (re-name appropriately)
adata.write(raw_file)


In [ ]:
### Normalize/library-size correct data and Log Transform
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
sc.pp.log1p(adata)


In [ ]:
### Store this version as raw data (Check this!!)
adata.raw=adata


In [ ]:
### Check anndata structure
adata


### Feature Selection

In [ ]:
### Find variable genes
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=8, min_disp=0.1)
sc.pl.highly_variable_genes(adata)


In [ ]:
### Filter genes
adata = adata[:, adata.var['highly_variable']]


### (optional) Regress covariates

In [ ]:
### Check cell-cycle genes
cell_cycle_genes = [x.strip() for x in open('regev_lab_cell_cycle_genes.txt')]

s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]
cell_cycle_genes = [x for x in cell_cycle_genes if x in adata.var_names]
sc.tl.score_genes_cell_cycle(adata,s_genes=s_genes,g2m_genes=g2m_genes)

cc_genes = adata[:,cell_cycle_genes]
sc.tl.pca(cc_genes)
sc.pl.pca_scatter(cc_genes,color='phase')


In [ ]:
### If necessary, regress out with counts and mito genes
sc.pp.regress_out(adata, ['n_counts', 'percent_mito'])


### Dimension Reduction

In [ ]:
### Scale data to unit variance, set max value
sc.pp.scale(adata, max_value=10)


In [ ]:
### Run PCA, find top PCs
sc.tl.pca(adata, svd_solver='arpack')
sc.pl.pca_variance_ratio(adata, log=True)


In [ ]:
### Compute tSNE coordinates using top PCs
sc.tl.tsne(adata,n_pcs = )


### Clustering
###### This will require a parameter sweep in the future

In [ ]:
### Compute nearest neighbors
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)


### Louvain Clustering, adjust resolution as needed
sc.tl.louvain(adata)

### Leiden Clustering, adjust resolution as needed
sc.tl.leiden(adata)



In [ ]:
### Plot Louvain clustering on UMAP coordinates
sc.pl.umap(adata, color=['louvain'])


In [ ]:
### Plot Leiden clustering on UMAP coordinates



In [ ]:
#### STOP point

### Cluster Labeling

In [ ]:
### T-Cell markers


In [ ]:
### B-Cell markers


In [ ]:
### Monocyte markers


In [ ]:
### NK markers


In [ ]:
### Other markers (DCs, etc.)


In [ ]:
### Label and Plot Cells
new_cluster_names = {'0':'CD4 T cells','1':'CD8 T cells','2':'CD8 T cells','3':'B cells','4':'NK cells','5,0':'monocytes','5,1':'DCs','5,2':'DCs','6':'NKT cells','7':'CD4 T cells','8':'unassigned'}
adata.obs['cell_type']=adata.obs['louvain_R'].map(new_cluster_names)

sc.pl.umap(adata, color=['louvain_R','cell_type'],legend_loc='on data',legend_fontsize=10)

### Save Everything Periodically

In [ ]:
adata.write(results_file)
